In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import typing as tp
import yaml
import random
import os
import sys
import soundfile as sf
import librosa
import cv2
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import resnest.torch as resnest_torch

from torchvision import models

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from radam import RAdam
from scipy.ndimage import maximum_filter1d
from tqdm import tqdm
import IPython

from scipy.io.wavfile import write
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [2]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
#     torch.backends.cudnn.deterministic = True  # type: ignore
#     torch.backends.cudnn.benchmark = True  # type: ignore
    

In [7]:
# ROOT = Path.cwd().parent
# INPUT_ROOT = ROOT / "input"
INPUT_ROOT = Path("/home/knikaido/work/Cornell-Birdcall-Identification/data")
RAW_DATA = INPUT_ROOT / "birdsong-recognition"
TRAIN_AUDIO_DIR = RAW_DATA / "train_audio"
# TRAIN_RESAMPLED_AUDIO_DIRS = [
#   INPUT_ROOT / "birdsong-resampled-train-audio-{:0>2}".format(i)  for i in range(5)
# ]
TEST_AUDIO_DIR = RAW_DATA / "test_audio"
TRAIN_RESAMPLED_AUDIO_DIRS = [
  INPUT_ROOT / "birdsong-resampled-train-audio-hpss-npz"
]


In [8]:
tmp_list = []
for audio_d in TRAIN_RESAMPLED_AUDIO_DIRS:
    if not audio_d.exists():
        continue
    for ebird_d in audio_d.iterdir():
        if ebird_d.is_file():
            continue
        for wav_f in ebird_d.iterdir():
            tmp_list.append([ebird_d.name, wav_f.name, wav_f.as_posix()])
            
train_wav_path_exist = pd.DataFrame(
    tmp_list, columns=["ebird_code", "resampled_filename", "file_path"])

del tmp_list

# train_all = pd.merge(
#     train, train_wav_path_exist, on=["ebird_code", "resampled_filename"], how="inner")

# print(train.shape)
print(train_wav_path_exist.shape)
# print(train_all.shape)

(21375, 3)


In [5]:
def my_makedirs(path):
    path = path.rsplit('/', 1)[0] + '/'
#     print(path)
    if not os.path.isdir(path):
        os.makedirs(path)

In [6]:
save_dir = '/home/knikaido/work/Cornell-Birdcall-Identification/data/birdsong-resampled-train-audio-hpss-npz/'
for i, file_name in tqdm(enumerate(train_wav_path_exist['file_path'])):
    try:
        y, sr = sf.read(file_name)
        save_path = save_dir + file_name.split('/', 7)[-1]
#         print(save_path[:-4])
#         break
        my_makedirs(save_path)
        np.save(save_path[:-4], y.astype('float32'))
    except:
        print('err!!!' + file_name)

21375it [2:01:58,  2.92it/s]


In [ ]:
save_dir = '/home/knikaido/work/Cornell-Birdcall-Identification/data/birdsong-resampled-train-audio-hpss-npz/'
for i, file_name in tqdm(enumerate(train_wav_path_exist['file_path'])):
    try:
        np.load(file_name)
    except:
        print('err!!!' + file_name)

778it [02:03,  6.32it/s]

err!!!/home/knikaido/work/Cornell-Birdcall-Identification/data/birdsong-resampled-train-audio-hpss-npz/houwre/XC232858.npy


818it [02:09,  6.33it/s]